In [1]:
import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('circuits'), '..')))

import argparse
import re
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import circuits.ht_circuit as ht
import qcm_sim as sim

# suppress qiskit 1.0 deprecation warnings:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

In [2]:

K = 6

for n in range(1,K+1):
    print("--------------------------------\n")
    print(f"Number of qubits: {n}\n")
    print("--------------------------------\n")

    # Create the hidden shift circuit
    qc = ht.ht_circuit(n)

    print(f"Quantum Circuit:\n")
    print(qc,"\n")

    # Simulate the circuit after decomposition
    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=4096)
    result = job.result()
    counts = result.get_counts()

    print("Qiskit Simulation Results:")
    #print(counts)
    print("Percentages: ",[x/sum(list(counts.values())) for x in list(counts.values())],"\n")
    print("--------------------------------\n")

    qasm_code = qc.qasm()

    # parent directory:
    pardir = os.path.abspath(os.path.join(os.path.dirname('circuits'), '..'))
    qasm_dir = os.path.join(pardir,'qasm_files')

    filename = f'ht_{n}.qasm'
    qasm_path = os.path.join(qasm_dir,filename)

    # Replace 'tdg' with 's; s; s; t;'
    modified_qasm_code = re.sub(r'\btdg\b', 's;\ns;\ns;\nt;', qasm_code)

    with open(qasm_path, "w") as f:
        f.write(modified_qasm_code)
    
    print("CNC Tableau Simulation Results:")

    # Specify the directory where the QASM file is located.
    filename = f"ht_{n}"
    qasm_file = filename + ".qasm"
    clifford_filename = filename + "_msi.qasm"

    # Run the gadgetized circuit simulation.
    counts, outputs, born_rule_estimates, shot_times = sim.run_qcm(qasm_dir, qasm_file, clifford_filename, shots=4096)

--------------------------------

Number of qubits: 1

--------------------------------

Quantum Circuit:

     ┌───┐┌───┐┌─┐
  q: ┤ T ├┤ H ├┤M├
     └───┘└───┘└╥┘
c: 1/═══════════╩═
                0  

Qiskit Simulation Results:
Percentages:  [0.512939453125, 0.487060546875] 

--------------------------------

CNC Tableau Simulation Results:
------------------------------
Initial conditions for simulation:
------------------------------

Negativity:
 1.0

------------------------------
Outputs of Simulation:
------------------------------

Average time per shot: 0.000141 seconds

Distinct Outcome: (0,)
Born Rule Estimate: 0.486572265625

Distinct Outcome: (1,)
Born Rule Estimate: 0.513427734375

--------------------------------

Number of qubits: 2

--------------------------------

Quantum Circuit:

     ┌───┐┌───┐┌───┐┌───┐┌─┐
  q: ┤ T ├┤ H ├┤ T ├┤ H ├┤M├
     └───┘└───┘└───┘└───┘└╥┘
c: 1/═════════════════════╩═
                          0  

Qiskit Simulation Results:
Percentages: